## Things to add
 - multi shot prompt 
 - chain of thought prompt 
 - saving responses 
 - chat memory (maybe)

# Model Testing 

**Here are my PC specs :)** 
 - CPU: AMD 5900x (12 core, 24 thread)
 - GPU: RTX 3080ti (12gb VRAM)
 - MEMORY: 32gb 3600mhz
 - STORAGE: 2 x 2TB NVME
---

In [ ]:
# import the stuff we need
import langchain
import ollama

## model selections 
 - Qwen 2.5 coder (0.5b) 
 - Qwen 2.5 coder (32b) 
 - Wizardcoder (33b)
 - Starcoder (1b)
 - Codellama (7b)
---


In [ ]:
# pick what model to use 
model1 = 'qwen2.5-coder:0.5b'
model2 = 'qwen2.5-coder:32b'
model3 = 'wizardcoder:33b'
model4 = 'starcoder2:15b'

## Prompt templates
 - one shot 
 - multi shot
 - chain of thought 
---

In [ ]:
# import the prompt block
from langchain.prompts import PromptTemplate

In [ ]:
oneShot = PromptTemplate(
    input_variables=["method_code"],
    template='''
Write a clear and professional method header in Javadoc style. I will provide some examples.

method_code: 
public int factorial(int n) {{
    if (n < 0) {{
        throw new IllegalArgumentException("Input must be non-negative.");
    }}
    int result = 1;
    for (int i = 2; i <= n; i++) {{
        result *= i;
    }}
    return result;
}}

good output:
/**
 * Calculates the factorial of a given non-negative integer.
 *
 * @param n the number to compute the factorial of
 * @return the factorial of n
 * @throws IllegalArgumentException if n is negative
 */

now given {method_code}:

write a Javadoc header for the method above.

'''
)


In [ ]:
from langchain.prompts import PromptTemplate
from ollama import chat

# Multishot examples from your dataset
examples = """
public Point getLoc(){return location;}

/***
* Gets the location.
*
* @return the current location
*/

---

public void draw(Graphics g){
    g.setColor(color);
    g.fillOval(location.intX() - radius, location.intY() - radius, radius * 2, radius * 2);
    if (!active){
        Vector standard_vec = move.normalize().scale(radius * 2);
        g.setColor(Color.RED);
        g.drawLine(location.intX(), location.intY(), standard_vec.move(location).intX(), standard_vec.move(location).intY());
    }
}

/***
* Draws the ball, and if non-active also draws its vector.
*
* @param g the Graphics context on which to draw
*/

---

public boolean isDone(){
    return finishedInstructions == totalInstructions;
}

/***
* Checks if the process is completed.
* 
* @return true, if the process has completed all instructions
*/
"""

# Method to document
new_method = """
private int add(int a, int b) {
    return a + b;
}
"""

# Build the prompt using LangChain
multiShot = PromptTemplate(
    input_variables=["examples", "new_code"],
    template="""
Here are some examples of Java methods and their Javadoc comments:

{examples}

Now write a professional Javadoc comment for this Java method:

{new_code}
"""
)

In [ ]:
prompt_text = multiShot.format(examples=examples, new_code=new_method)

# Run the model
response = chat(model="qwen2.5-coder:0.5b", messages=[{'role': 'user', 'content': prompt_text}])
print(response['message']['content'])

In [ ]:
chainOfThought = PromptTemplate(
    template='''
Write to me a professional javadoc for this Java funcion. Here are the steps you should take:
Is this method for a data structure? If so, what data structure?
What are the parameters?
What does the method do? Look at the name of the method
How does the method do what it does? Are there edge cases?
What exceptions does the method throw?
What is the state of the object before the method is called?
What is the state of the object after the method is called?
What does the method return?
Is this method overriding another method?
Once you have finished the analysis, generate a professional javadoc with a clear description of what the method does, parameters taken in, return values, pre and post conditions, exceptions, and any other notes that should be in the javadoc. 
'''
)

In [ ]:
AdvancedChainOfThought = PromptTemplate(
    template='''
Write to me a professional javadoc for this Java funcion. Here are the steps you should take:
Is this method for a data structure? If so, what data structure?
What are the parameters?
What does the method do? Look at the name of the method
How does the method do what it does?
What are the variable names used?
How are the variables used?
What exceptions does the method throw?
What inputs will cause an exception to be thrown?
Trace the execution with some test runs
If the method is complex, break it into smaller chunks and analyze each chuck to see what the method does
Once you have finished the analysis, generate a professional javadoc with a clear description of what the method does, parameters taken in, return values, pre and post conditions, exceptions, and any other notes that should be in the javadoc. 
'''
)

## Sample code 
---

In [ ]:
removeFromBST = oneShot.format(method_code=
    """private Node doRemove(Node r, String s, Node before, boolean removeOnce, Integer val) {
		if(r == null) {
			lastRemoved = null;
			return null;
		}
		int c = s.compareTo(r.string);
		if(c < 0) r.left = doRemove(r.left, s, before, removeOnce, val);
		else if(c > 0) r.right = doRemove(r.right, s, r, removeOnce, val);
		else {
			if(val != null && val != r.count) return r;
			lastRemoved = r.count;
			if(removeOnce && r.count > 1) {
				r.count--;
				return r;
			}
			if(before.next != r) {
				before = r.left;
				while(before.right != null) before = before.right;
			}
			version++;
			numEntries--;
			if(r.left == null) {
				before.next = r.next;
				return r.right;
			}
			if(r.right == null) {
				before.next = r.next;
				return r.left;
			}
			
			Node successor = r.right;
			Node prev = r;
			while(successor.left != null) {
				prev = successor;
				successor = successor.left;
			}
			r.string = successor.string;
			r.count = successor.count;
			
			if(prev.left == successor) prev.left = successor.right;
			else prev.right = successor.right;
			
			r.next = successor.next;


		}
		return r;
"""
)

In [ ]:
# add some content 
shortMethod = oneShot.format(
    method_code = """
    private void connect(HexPiece p) {
HexCoordinate h = p.getLocation();
for (HexDirection d : HexDirection.values()) {
HexCoordinate h2 = d.move(h);
HexPiece p2 = findPiece(h2);
if (p2 != null) {
p.neighbors[d.ordinal()] = p2;
p2.neighbors[d.reverse().ordinal()] = p;
}
}
}
                         """)

In [ ]:
longMethodWithHelper = oneShot.format(method_code="""\
private void rehash() {{

    int newCapacity = Primes.nextPrime(table.length * 2);
    HexPiece[] oldTable = table;
    table = new HexPiece[newCapacity];
    size = 0;

    for (HexPiece head : oldTable) {{
        if (head == null) continue;
        HexPiece current = head;
        do {{
            HexPiece next = current.nextInChain;
            int newIndex = locate(current.location);

            if (table[newIndex] == null) {{
                current.nextInChain = current;
                table[newIndex] = current;
            }} else {{
                current.nextInChain = table[newIndex].nextInChain;
                table[newIndex].nextInChain = current;
            }}
            size++;
            current = next;
        }} while (current != head);
    }}
}}

public boolean add(HexPiece p) {{
    assert wellFormed() : "Invariant broken before add";

    if (p == null || p.location == null || p.terrain == null)
        throw new NullPointerException("Piece, location, and terrain must not be null");

    int index = locate(p.location);
    HexPiece head = table[index];

    if (head != null) {{
        HexPiece current = head;
        do {{
            if (current.location.equals(p.location)) {{
                current.terrain = p.terrain;
                version++;
                assert wellFormed() : "Invariant broken after add (update)";
                return false;
            }}
            current = current.nextInChain;
        }} while (current != head);

        p.nextInChain = head.nextInChain;
        head.nextInChain = p;
    }} else {{
        p.nextInChain = p;
        table[index] = p;
    }}

    connect(p);
    size++;
    version++;

    if (size >= table.length) {{
        rehash();
    }}

    assert wellFormed() : "Invariant broken after add (insert)";
    return true;
}}
""")


# Evaluating outputs

### Qwen2.5 coder (0.5b)

In [ ]:
# Run model
from ollama import chat
model1 = 'qwen2.5-coder:0.5b'
response = chat(model=model1, messages=[{'role': 'user', 'content': removeFromBST}])
print(response['message']['content'])


### Qwen 2.5 coder (32b)

In [ ]:
# Run model
response = chat(model=model2, messages=[{'role': 'user', 'content': longMethodWithHelper}])
print(response['message']['content'])


### Wizard Coder (15B)

In [ ]:
# Run model
response = chat(model=model3, messages=[{'role': 'user', 'content': longMethodWithHelper}])
print(response['message']['content'])

### Starcoder2 (15b)

In [ ]:
# Run model
response = chat(model=model4, messages=[{'role': 'user', 'content': shortMethod}])
print(response['message']['content'])